In [1]:
import numpy as np
import pandas as pd
import glob

In [2]:
state_codes = [
    53, # Washington
    41, # Oregon
    6,  # California
    32, # Nevada
    16, # Idaho
    56, # Wyoming
    30, # Montana
    35, # New Mexico
    4,  # Arizona
    8,  # Colorado
    49  # Utah
]
mon_loc = pd.read_csv('PM_new_coord.csv', dtype={'Grid_Cell':'Int64', 'ID':'Int64'})
print(mon_loc.head)

<bound method NDFrame.head of      Grid_Cell          pm_x          pm_y     pm_lat      pm_lon   ID  \
0        15380 -1.545579e+06 -8.821908e+05  31.337204 -110.936718    1   
1        17341 -1.412537e+06 -9.079196e+05  31.349200 -109.539683    2   
2        21429 -1.137201e+06 -8.923216e+05  31.787885 -106.683324    3   
3        21607 -1.125669e+06 -8.930981e+05  31.796218 -106.584434    4   
4        13060 -1.698603e+06 -7.817011e+05  31.950000 -112.801000    5   
..         ...           ...           ...        ...         ...  ...   
475      16611 -1.424518e+06  1.034871e+06  48.510300 -113.996810  476   
476       7267 -2.042921e+06  1.208466e+06  48.520590 -122.614280  477   
477      12319 -1.708344e+06  1.104896e+06  48.544722 -117.903611  478   
478       9080 -1.924266e+06  1.192342e+06  48.731420 -121.065815  479   
479       7632 -2.017071e+06  1.221384e+06  48.762780 -122.440280  480   

     Unnamed: 6  
0           NaN  
1           NaN  
2           NaN  
3        

In [9]:
first = True
pm_df = None
for fn in glob.glob('.\Daily\*.csv'):
    print(fn)
    df = pd.read_csv(fn, usecols=['State Code', 'County Code', 'Site Num', 'Date Local', 'Latitude', 'Longitude', 'Arithmetic Mean', 'Sample Duration'])
    df = df[df['State Code'].isin(state_codes)]
    
    if first:
        pm_df = df
        first = False
    else:
        pm_df = pm_df.append(df)
        
for fn in glob.glob('.\Hourly\*.csv'):
    print(fn)
    df = pd.read_csv(fn, usecols=['State Code', 'Date Local', 'Latitude', 'Longitude', 'Sample Measurement', 'Method Name', 'Time Local'])
    df = df[df['State Code'].isin(state_codes)]
    
    pm_df = pm_df.append(df)
    


.\Daily\daily_88101_2013.csv
.\Daily\daily_88101_2014.csv
.\Daily\daily_88502_2013.csv
.\Daily\daily_88502_2014.csv
.\Hourly\hourly_88101_2013.csv
.\Hourly\hourly_88101_2014.csv
.\Hourly\hourly_88502_2013.csv
.\Hourly\hourly_88502_2014.csv


In [27]:
def r(g):
    return g[['Latitude', 'Longitude']].loc[[g.index[0]]]

gs = pm_df.groupby(['Latitude', 'Longitude'])
locs = gs.apply(r)

print(locs.head)
locs.to_csv('mon_loc.csv', index=False)

<bound method NDFrame.head of                                Latitude   Longitude
Latitude  Longitude                                
31.337204 -110.936718 15790   31.337204 -110.936718
31.349200 -109.539683 7963    31.349200 -109.539683
31.787885 -106.683324 165512  31.787885 -106.683324
31.796218 -106.584434 164797  31.796218 -106.584434
31.950000 -112.801000 8894    31.950000 -112.801000
...                                 ...         ...
48.510300 -113.996810 150908  48.510300 -113.996810
48.520590 -122.614280 301385  48.520590 -122.614280
48.544722 -117.903611 329631  48.544722 -117.903611
48.731420 -121.065815 331764  48.731420 -121.065815
48.762780 -122.440280 304898  48.762780 -122.440280

[480 rows x 2 columns]>


In [15]:
# parse daily data
first = True
pm_df = None
for fn in glob.glob('.\Daily\*.csv'):
    print(fn)
    df = pd.read_csv(fn, usecols=['State Code', 'Date Local', 'Latitude', 'Longitude', 'Arithmetic Mean', 'Sample Duration', 'Parameter Code'])
    df = df[df['State Code'].isin(state_codes)]
    df = pd.merge(mon_loc, df, how='inner', left_on=['pm_lat', 'pm_lon'], right_on=['Latitude', 'Longitude'])
    
    df.rename(columns={'Grid_Cell':'gridID', 'ID':'monID', 'Arithmetic Mean':'pm25', 'Date Local':'date'}, inplace=True)
    
    if first:
        pm_df = df
        first = False
    else:
        pm_df = pm_df.append(df)
        


print(pm_df.shape)
print(pm_df.head)
print(pd.unique(pm_df['monID']))
print(pd.unique(pm_df['Sample Duration']))

.\Daily\daily_88101_2013.csv
.\Daily\daily_88101_2014.csv
.\Daily\daily_88502_2013.csv
.\Daily\daily_88502_2014.csv
(471811, 14)
<bound method NDFrame.head of         gridID          pm_x          pm_y     pm_lat      pm_lon  monID  \
0        15380 -1.545579e+06 -8.821908e+05  31.337204 -110.936718      1   
1        15380 -1.545579e+06 -8.821908e+05  31.337204 -110.936718      1   
2        15380 -1.545579e+06 -8.821908e+05  31.337204 -110.936718      1   
3        15380 -1.545579e+06 -8.821908e+05  31.337204 -110.936718      1   
4        15380 -1.545579e+06 -8.821908e+05  31.337204 -110.936718      1   
...        ...           ...           ...        ...         ...    ...   
114218    9080 -1.924266e+06  1.192342e+06  48.731420 -121.065815    479   
114219    9080 -1.924266e+06  1.192342e+06  48.731420 -121.065815    479   
114220    9080 -1.924266e+06  1.192342e+06  48.731420 -121.065815    479   
114221    9080 -1.924266e+06  1.192342e+06  48.731420 -121.065815    479   
11422

In [16]:
def fn(group):
    mode = group.loc[group['Parameter Code'] == 88101]
    if len(mode) >= 1:
        a = mode.loc[mode['Sample Duration'] == '24 HOUR']
        if len(a) >= 1:
            res = a.iloc[[0]].drop(columns=['date', 'monID']).copy()
            res['priority'] = 6
            return res
        b = mode.loc[mode['Sample Duration'] == '24-HR BLK AVG']
        if len(b) >= 1:
            res = b.iloc[[0]].drop(columns=['date', 'monID']).copy()
            res['priority'] = 5
            return res
        c = mode.loc[mode['Sample Duration'] == '1 HOUR']
        if len(c) >= 1:
            res = c.iloc[[0]].drop(columns=['date', 'monID']).copy()
            res['priority'] = 4
            return res
    
    mode = group.loc[group['Parameter Code'] == 88502]
    a = mode.loc[mode['Sample Duration'] == '24 HOUR']
    if len(a) >= 1:
        res = a.iloc[[0]].drop(columns=['date', 'monID']).copy()
        res['priority'] = 3
        return res
    b = mode.loc[mode['Sample Duration'] == '24-HR BLK AVG']
    if len(b) >= 1:
        res = b.iloc[[0]].drop(columns=['date', 'monID']).copy()
        res['priority'] = 2
        return res
    c = mode.loc[mode['Sample Duration'] == '1 HOUR']
    if len(c) >= 1:
        res = c.iloc[[0]].drop(columns=['date', 'monID']).copy()
        res['priority'] = 1
        return res
    

pm_df_grouped = pm_df.groupby(['date', 'monID'])
print(len(pm_df_grouped))
pm_df = pm_df_grouped.apply(fn).reset_index()

print(pm_df.head)
print(pm_df['Sample Duration'].value_counts())

225816
<bound method NDFrame.head of               date  monID  level_2  gridID          pm_x          pm_y  \
0       2013-01-01      2        0   17341 -1.412537e+06 -9.079196e+05   
1       2013-01-01      3      476   21429 -1.137201e+06 -8.923216e+05   
2       2013-01-01      4     1201   21607 -1.125669e+06 -8.930981e+05   
3       2013-01-01      5     1555   13060 -1.698603e+06 -7.817011e+05   
4       2013-01-01      6     2024   21609 -1.124457e+06 -8.685132e+05   
...            ...    ...      ...     ...           ...           ...   
225811  2014-12-31    475   113293   26887 -7.307322e+05  9.212361e+05   
225812  2014-12-31    476   113413   16611 -1.424518e+06  1.034871e+06   
225813  2014-12-31    477   121540    7267 -2.042921e+06  1.208466e+06   
225814  2014-12-31    479   114222    9080 -1.924266e+06  1.192342e+06   
225815  2014-12-31    480   122204    7632 -2.017071e+06  1.221384e+06   

           pm_lat      pm_lon  Unnamed: 6  State Code  Parameter Code  \
0

In [18]:
def hourly_mean(g):
    mean = g['Sample Measurement'].mean()
    res = g.iloc[[0]].copy()
    res['Sample Measurement'] = mean
    if res['Parameter Code'].iloc[0] == 88101:
        res['priority'] = 8
    else:
        res['priority'] = 7
    return res.drop(columns=['Date Local', 'Latitude', 'Longitude'])

# parse hourly data
for fn in glob.glob('.\Hourly\*.csv'):
    print(fn)
    df = pd.read_csv(fn, usecols=['State Code', 'Date Local', 'Latitude', 'Longitude', 'Sample Measurement', 'Time Local', 'Parameter Code'])
    df = df[df['State Code'].isin(state_codes)]
    grouped = df.groupby(['Date Local', 'Latitude', 'Longitude'])
    #df = grouped[['Sample Measurement']].mean().reset_index()
    df = grouped.apply(hourly_mean).reset_index()
    df = pd.merge(mon_loc, df, how='inner', left_on=['pm_lat', 'pm_lon'], right_on=['Latitude', 'Longitude'])
    df.rename(columns={'Grid_Cell':'gridID', 'ID':'monID', 'Sample Measurement':'pm25', 'Date Local':'date'}, inplace=True)
    
    pm_df = pm_df.append(df)


print(pd.unique(pm_df['monID']))
print(pm_df.shape)

.\Hourly\hourly_88101_2013.csv
.\Hourly\hourly_88101_2014.csv
.\Hourly\hourly_88502_2013.csv
.\Hourly\hourly_88502_2014.csv
<IntegerArray>
[  2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
 ...
 360, 356, 171,  22, 378, 159,  29,  23,  31, 313]
Length: 480, dtype: Int64
(413904, 18)


In [ ]:
def sizing(group):
    l = len(group)
    if l != 24:
        return group.join(pd.Series([l]*l, name='size', index=group.index))
    
    
grouped = pm_df.groupby(['date', 'monID'])
sizes = grouped.apply(sizing)
sizes = sizes.loc[sizes['size'] != 24]
sizes.to_csv('hourly_anomaly.csv', columns=['date', 'monID', 'Time Local', 'Method Name', 'pm25', 'size'], index=False)

In [36]:
def take(g):
    return g.loc[[g['priority'].idxmax()]].drop(columns=['monID', 'date'])

print(pm_df.shape)

pm_df_grouped = pm_df.groupby(['monID', 'date'])
pm_df = pm_df_grouped.apply(take).drop(columns=['level_2']).reset_index()


print(pd.unique(pm_df['monID']))
print(len(pd.unique(pm_df['monID'])))
print(pm_df.shape)

(413904, 18)
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 2

In [53]:
def r(g):
    monIDs = pd.unique(g['monID'])
    if len(monIDs) == 1:
        return g.drop(columns=['gridID'])
    print(monIDs)
    mon_groups = g.groupby(['monID'])
    max_datapoints = 0
    highest_priority = 0
    best = None
    for _, group in mon_groups:
        if len(group) > max_datapoints:
            best = group
            max_datapoints = len(group)
            highest_priority = 0
        elif len(group) == max_datapoints:
            priority = group['priority'].mean()
            if priority > highest_priority:
                best = group
                highest_priority = priority
    
    return best.drop(columns=['gridID'])
    

g = pm_df.groupby(['gridID'])
print(len(g))
pm_df_unique = g.apply(r).reset_index()
print(pm_df_unique.shape)
print(sorted(pd.unique(pm_df_unique['monID'])))
pm_df_unique.to_csv('pm25.csv', columns=['date', 'gridID', 'monID', 'pm25', 'Parameter Code'], index=False)

429
[274 277]
[201 202]
[158 159]
[188 189]
[104 105]
[319 320]
[205 206]
[331 332]
[335 336]
[312 313 314]
[ 97  99 101]
[114 115]
[244 245]
[378 380 381]
[56 57 58]
[213 214]
[233 234]
[340 341]
[78 79]
[424 426 427]
[441 442 445 448 449]
[31 32]
[14 17]
[66 67]
[459 460]
[20 22]
[29 30]
[23 24]
[435 436]
[422 423]
[133 134]
[368 369]
[395 396]
[399 401]
[41 46]
[36 37]
[38 39]
[10 12]
[273 275]
[107 108]
[239 240 243 246]
(206773, 19)
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 15, 16, 17, 18, 19, 20, 21, 24, 25, 26, 27, 28, 30, 32, 33, 34, 35, 37, 38, 40, 41, 42, 43, 44, 45, 47, 48, 49, 50, 51, 52, 53, 54, 55, 57, 59, 60, 61, 62, 63, 64, 65, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 98, 100, 101, 102, 103, 105, 106, 108, 109, 110, 111, 112, 113, 114, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 

           monID
pm25            
1     250.657596
